In [2]:
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-57d99ce9-5553-0f30-b547-3062373aa407)


In [4]:
import pandas as pd

df = pd.read_csv("data/siyaset.csv")
df.head()

,category,text
0,siyaset,3 milyon ile ön seçim vaadi mhp nin 10 olağan...
1,siyaset,mesut_yılmaz yüce_divan da ceza alabilirdi pr...
2,siyaset,disko lar kaldırılıyor başbakan_yardımcısı ar...
3,siyaset,sarıgül anayasa_mahkemesi ne gidiyor mustafa_...
4,siyaset,erdoğan idamın bir haklılık sebebi var demek ...


In [5]:
df.category.value_counts()

kultur        700
teknoloji     700
ekonomi       700
spor          700
saglik        700
dunya         700
siyaset       700
Name: category, dtype: int64

In [6]:
# Get train sentences from df.comment
train_sentences = df["text"].tolist()

train_sentences[0]

' 3 milyon ile ön seçim vaadi mhp nin 10 olağan büyük kurultayı nda konuşan genel başkan adayı koray_aydın seçimlerden önce partinin üye sayısının 3 milyona ulaştırılması hedefini koyarak ön seçim uygulaması vaadinde bulundu mhp nin 10 olağan büyük kurultayı nda konuşan genel başkan adayı koray_aydın seçimlerden önce partinin üye sayısının 3 milyona ulaştırılması hedefini koyarak ön seçim uygulaması vaadinde bulundu genel_başkan adayı koray_aydın kürsüye beklenirken yapılan tezahüratlar ve ıslıklamalar üzerine divan başkanı tuğrul_türkeş mhp nin genel başkanlığı da genel başkan adaylığı da saygıdeğer işlerdir bu salondaki herkes ciddiye almak zorundadır dedi ve taşkınlıklara izin verilmeyeceğini salonda sükunet sağlanmadan konuşmaların başlamayacağını vurguladı türkeş devlet_bahçeli nin kurultay açılışında konuştuğu için adaylık nedeniyle ikinci bir konuşma yapmayacağını açıkladı konuşmasında kurultayın mhp nin tek başına iktidarına vesile olmasını dileyen aydın ak_parti nin mhp yi ele

In [7]:
# Get the average and the max length of the inputs
import numpy as np

sent_lens = [len(sentence.split()) for sentence in train_sentences]
avg_sent_len = np.mean(sent_lens)
max_sent_len = np.max(sent_lens)
avg_sent_len, max_sent_len

(268.6981632653061, 2711)

In [8]:
import numpy as np
output_seq_len = int(np.percentile(sent_lens, 95))

output_seq_len

673

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-turkish-128k-uncased', do_lower_case=True)
sentences = df.text.values
max_len = 250

Downloading:   0%|          | 0.00/1.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/386 [00:00<?, ?B/s]

In [10]:
import tensorflow as tf

input_ids = []
attention_mask = []

for txt in df.text.values:
    encoded = tokenizer.encode_plus(
        text=txt, # the sentence to be encoded 
        add_special_tokens=True, # Add [CLS] and [SEP]
        max_length=250, # max length of a sentence
        truncation=True, # truncate if sentence length is bigger than max_length
        pad_to_max_length=True, # Add [PAD]s
        return_attention_mask=True, # Generate attention mask
        return_tensors="tf" # return TenforFlow tensors
    )

    # Append input_ids and attention_masks to their own lists
    input_ids.append(encoded["input_ids"])
    attention_mask.append(encoded["attention_mask"])

# Concatenate
input_ids = tf.concat(input_ids, 0)
attention_mask = tf.concat(attention_mask, 0)

print("Original: ", df.text.values[0])
print("Token IDs: ", input_ids[0])

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:   3 milyon ile ön seçim vaadi mhp nin 10 olağan büyük kurultayı nda konuşan genel başkan adayı koray_aydın seçimlerden önce partinin üye sayısının 3 milyona ulaştırılması hedefini koyarak ön seçim uygulaması vaadinde bulundu mhp nin 10 olağan büyük kurultayı nda konuşan genel başkan adayı koray_aydın seçimlerden önce partinin üye sayısının 3 milyona ulaştırılması hedefini koyarak ön seçim uygulaması vaadinde bulundu genel_başkan adayı koray_aydın kürsüye beklenirken yapılan tezahüratlar ve ıslıklamalar üzerine divan başkanı tuğrul_türkeş mhp nin genel başkanlığı da genel başkan adaylığı da saygıdeğer işlerdir bu salondaki herkes ciddiye almak zorundadır dedi ve taşkınlıklara izin verilmeyeceğini salonda sükunet sağlanmadan konuşmaların başlamayacağını vurguladı türkeş devlet_bahçeli nin kurultay açılışında konuştuğu için adaylık nedeniyle ikinci bir konuşma yapmayacağını açıkladı konuşmasında kurultayın mhp nin tek başına iktidarına vesile olmasını dileyen aydın ak_parti nin 

In [11]:
# Convert tokens to ids to check
x = tokenizer.convert_ids_to_tokens(input_ids[0])
x

['[CLS]',
 '3',
 'milyon',
 'ile',
 'on',
 'secim',
 'vaadi',
 'mhp',
 'nin',
 '10',
 'olagan',
 'buyuk',
 'kurultayı',
 'nda',
 'konus',
 '##an',
 'genel',
 'baskan',
 'adayı',
 'koray',
 '_',
 'aydın',
 'secim',
 '##lerden',
 'once',
 'partinin',
 'uye',
 'sayısının',
 '3',
 'milyona',
 'ulast',
 '##ırı',
 '##lması',
 'hedefini',
 'koyarak',
 'on',
 'secim',
 'uygulaması',
 'vaadinde',
 'bulundu',
 'mhp',
 'nin',
 '10',
 'olagan',
 'buyuk',
 'kurultayı',
 'nda',
 'konus',
 '##an',
 'genel',
 'baskan',
 'adayı',
 'koray',
 '_',
 'aydın',
 'secim',
 '##lerden',
 'once',
 'partinin',
 'uye',
 'sayısının',
 '3',
 'milyona',
 'ulast',
 '##ırı',
 '##lması',
 'hedefini',
 'koyarak',
 'on',
 'secim',
 'uygulaması',
 'vaadinde',
 'bulundu',
 'genel',
 '_',
 'baskan',
 'adayı',
 'koray',
 '_',
 'aydın',
 'kursu',
 '##ye',
 'beklenirken',
 'yapılan',
 'tezah',
 '##urat',
 '##lar',
 've',
 'ıslık',
 '##lamalar',
 'uzerine',
 'divan',
 'baskan',
 '##ı',
 'tug',
 '##rul',
 '_',
 'turk',
 '##es',
 

In [12]:
tokenizer.convert_tokens_to_ids(x)

[2,
 23,
 2961,
 2037,
 2121,
 75030,
 47568,
 6321,
 2276,
 2562,
 110065,
 20086,
 30418,
 3334,
 5484,
 1928,
 2468,
 34449,
 6892,
 20249,
 41,
 3948,
 75030,
 2884,
 18094,
 10349,
 84087,
 8369,
 23,
 18462,
 114961,
 3940,
 120685,
 28262,
 14540,
 2121,
 75030,
 6394,
 87715,
 4502,
 6321,
 2276,
 2562,
 110065,
 20086,
 30418,
 3334,
 5484,
 1928,
 2468,
 34449,
 6892,
 20249,
 41,
 3948,
 75030,
 2884,
 18094,
 10349,
 84087,
 8369,
 23,
 18462,
 114961,
 3940,
 120685,
 28262,
 14540,
 2121,
 75030,
 6394,
 87715,
 4502,
 2468,
 41,
 34449,
 6892,
 20249,
 41,
 3948,
 6905,
 3356,
 37662,
 2649,
 17852,
 90680,
 1934,
 1946,
 48998,
 10086,
 45898,
 11835,
 34449,
 1022,
 36527,
 9021,
 41,
 6142,
 1951,
 6321,
 2276,
 2468,
 34449,
 49313,
 1972,
 2468,
 34449,
 3554,
 49313,
 1972,
 5316,
 1942,
 4829,
 72798,
 2025,
 1964,
 51135,
 3171,
 15818,
 3632,
 10121,
 2841,
 1946,
 80525,
 1930,
 21291,
 3794,
 35219,
 98117,
 4389,
 17367,
 27101,
 13224,
 1025,
 9598,
 25110,


In [13]:
# Check input_ids and shape of input_ids
input_ids, input_ids.shape

(<tf.Tensor: shape=(4900, 250), dtype=int32, numpy=
 array([[     2,     23,   2961, ...,  56166,  36859,      3],
        [     2,  11633,     41, ...,  22328,  80447,      3],
        [     2,  50235,   4861, ...,     24,   4642,      3],
        ...,
        [     2,   6865,   2370, ...,   2351,   1964,      3],
        [     2, 123116,  30632, ...,  44145,  46950,      3],
        [     2,   5533,   1964, ...,      0,      0,      0]], dtype=int32)>,
 TensorShape([4900, 250]))

In [14]:
# Check attention_mask and shape of attention_mask
attention_mask, attention_mask.shape

(<tf.Tensor: shape=(4900, 250), dtype=int32, numpy=
 array([[1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        ...,
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 1, 1, 1],
        [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>, TensorShape([4900, 250]))

In [15]:
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
labels_one_hot = one_hot_encoder.fit_transform(df["category"].to_numpy().reshape(-1,1))
labels_one_hot

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [16]:
dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, labels_one_hot))
dataset.take(1)

<TakeDataset shapes: ((250,), (250,), (7,)), types: (tf.int32, tf.int32, tf.float64)>

In [17]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {"input_ids": input_ids,
            "attention_mask": masks}, labels

In [18]:
dataset = dataset.map(map_func)
dataset.take(1)

<TakeDataset shapes: ({input_ids: (250,), attention_mask: (250,)}, (7,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [19]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

x = len(dataset)*0.85

num_epochs = 3
num_train_steps = x * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5,
    end_learning_rate=0.,
    decay_steps=num_train_steps
)

In [20]:
batch_size=16
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)

dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 250), attention_mask: (16, 250)}, (16, 7)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [21]:
input_ids[0], input_ids[0].shape

(<tf.Tensor: shape=(250,), dtype=int32, numpy=
 array([     2,     23,   2961,   2037,   2121,  75030,  47568,   6321,
          2276,   2562, 110065,  20086,  30418,   3334,   5484,   1928,
          2468,  34449,   6892,  20249,     41,   3948,  75030,   2884,
         18094,  10349,  84087,   8369,     23,  18462, 114961,   3940,
        120685,  28262,  14540,   2121,  75030,   6394,  87715,   4502,
          6321,   2276,   2562, 110065,  20086,  30418,   3334,   5484,
          1928,   2468,  34449,   6892,  20249,     41,   3948,  75030,
          2884,  18094,  10349,  84087,   8369,     23,  18462, 114961,
          3940, 120685,  28262,  14540,   2121,  75030,   6394,  87715,
          4502,   2468,     41,  34449,   6892,  20249,     41,   3948,
          6905,   3356,  37662,   2649,  17852,  90680,   1934,   1946,
         48998,  10086,  45898,  11835,  34449,   1022,  36527,   9021,
            41,   6142,   1951,   6321,   2276,   2468,  34449,  49313,
          1972,  

In [22]:
split = 0.85
size = int((input_ids.shape[0] / batch_size) * split)
size

260

In [23]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

len(dataset), len(train_ds), len(val_ds)

(306, 260, 46)

In [24]:
from transformers import TFAutoModel
from transformers import BertTokenizer

bert128k = TFAutoModel.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")
#bert = TFAutoModel.from_pretrained("dbmdz/bert-base-turkish-uncased")
#distilbert = TFAutoModel.from_pretrained("dbmdz/distilbert-base-turkish-cased")

Downloading:   0%|          | 0.00/1.06G [00:00<?, ?B/s]

Some layers from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [26]:
import tensorflow as tf

# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(250,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(250,), name='attention_mask', dtype='int32')

# we access the transformer model within our bert object using the bert attribute (eg bert.bert instead of bert)
embeddings = bert128k.bert(input_ids, attention_mask=mask)[1]  # access final activations (already max-pooled) [1]
# convert bert embeddings into 6 output classes
#x = tf.keras.layers.Dropout(0.1)(embeddings)
x = tf.keras.layers.Dense(1024, activation='relu')(embeddings)
outputs = tf.keras.layers.Dense(7, activation='softmax', name='outputs')(x)

# model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=outputs)

In [27]:
model.layers[2].trainable=False
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 250)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 250)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  184345344   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 250,                                           

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler) # lr=2e-5, decay=1e-6
loss = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')

model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=[acc])

In [29]:
train_ds, val_ds

(<TakeDataset shapes: ({input_ids: (16, 250), attention_mask: (16, 250)}, (16, 7)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>,
 <SkipDataset shapes: ({input_ids: (16, 250), attention_mask: (16, 250)}, (16, 7)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>)

In [30]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=4,
    verbose=1
)

Epoch 1/4
260/260 [==============================] - 187s 685ms/step - loss: 1.5426 - accuracy: 0.4882 - val_loss: 1.1468 - val_accuracy: 0.7242
Epoch 2/4
260/260 [==============================] - 175s 675ms/step - loss: 1.0414 - accuracy: 0.7024 - val_loss: 0.8548 - val_accuracy: 0.7840
Epoch 3/4
260/260 [==============================] - 175s 674ms/step - loss: 0.8447 - accuracy: 0.7536 - val_loss: 0.7674 - val_accuracy: 0.7649
Epoch 4/4
260/260 [==============================] - 176s 678ms/step - loss: 0.7628 - accuracy: 0.7630 - val_loss: 0.6646 - val_accuracy: 0.8152
